In [3]:
#Align and crop face
import os
import cv2
import pandas as pd
from helpers.face import Face
from helpers.image_reader import read_image
from preprocessors.asymmetry_calculator import process_all_face
def ensure_directory_exists(directory):
    #Ensure that a given directory exists.
    if not os.path.exists(directory):
        os.mkdir(directory)
        
group_directories = {'PD','HC'}
group_directory = 'PD'
output_feature_folder = f'./data/working/csv-features/'
output_single_folder = f'./data/working/csv-single/'
ensure_directory_exists(output_single_folder)
version = 'v3'

In [47]:
import pandas as pd
import os
import numpy as np

def process_and_aggregate_files(output_feature_folder,folder_name, output_file,samples, label):
    aggregated_data = []  # List to store all data frames before concatenation
    folder_name = os.path.join(output_feature_folder, folder_name)
    # Iterate through each file in the directory
    for filename in os.listdir(folder_name):
        if filename.endswith('.csv'):
            # Construct the full file path
            file_path = os.path.join(folder_name, filename)
            
            # Read the CSV file
            data = pd.read_csv(file_path)
            print(f"Processing: {filename}")
            # Randomly select 5 rows
            if len(data) > samples:
                #data_sampled = data.sample(samples, random_state=42)
                data_sampled = data.head(samples)
            else:
                data_sampled = data
            
            # Modify the data by adding the new first column and removing the actual first column
            data_sampled.insert(0, 'Person_Data', filename.split('.')[0] + '_' + data_sampled.iloc[:, 0].astype(str))
            data_sampled.drop(data_sampled.columns[1], axis=1, inplace=True)
            
            # Add label column if needed
            if label is not None:
                data_sampled['Label'] = label
            
            # Append the processed DataFrame to the list
            aggregated_data.append(data_sampled)
    
    # Concatenate all DataFrames in the list into one DataFrame
    final_data = pd.concat(aggregated_data, ignore_index=True)
    
    # Save the aggregated data to a CSV file
    final_data.to_csv(output_file, index=False)
    print(f"Aggregated and saved: {output_file}")

samples=1
# Aggregate and process files for both HC and PD folders
parkinson_file = os.path.join(output_single_folder,f'PD-{samples}-output.csv')
healthy_file = os.path.join(output_single_folder,f'HC-{samples}-output.csv')
process_and_aggregate_files(output_feature_folder,'HC', healthy_file,samples, 0)
process_and_aggregate_files(output_feature_folder,'PD', parkinson_file,samples, 1)


Processing: 00c84508-07d2-5a70-65da-a3b6a5cf711b-joy-output.csv
Processing: 036dfb9b-b3e1-d827-acb4-c37dae386d9d-joy-output.csv
Processing: 07571_happy-output.csv
Processing: 07580_happy-output.csv
Processing: 07609_happy-output.csv
Processing: 07610_Happy-output.csv
Processing: 07629_Happy-output.csv
Processing: 07652_happy-output.csv
Processing: 07735_Happy-output.csv
Processing: 07794_Happy-output.csv
Processing: 07900_Happy-output.csv
Processing: 07928_Happy-output.csv
Processing: 0a7b79dc-d9d0-5364-8452-bdb432302c49-joy-output.csv
Processing: 0e056939-a09c-7951-0a6f-a51bfaa6d679-joy-output.csv
Processing: 0e4bd8d3-b051-a199-82c0-d8e7152a66ba-joy-output.csv
Processing: 1295b5b3-5dda-1d6d-96b3-7d732d60c1f5-joy-output.csv
Processing: 13fc5999-8f79-1b7f-d55b-7c7b8a1a8fcc-joy-output.csv
Processing: 155b4a46-9d11-b1e7-32c1-a5a7892cd278-joy-output.csv
Processing: 16339d27-6db5-f08f-0116-aa16616f6340-joy-output.csv
Processing: 1f985eb2-1b36-c727-eb8b-e1fbcc62f3d5-joy-output.csv
Processing

In [15]:
from sklearn.discriminant_analysis import StandardScaler
from xgboost import XGBClassifier
from helpers.gridsearch_classifier import gridsearch_classifiers_group
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import GroupKFold, train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import joblib
from sklearn.discriminant_analysis import StandardScaler
from sklearn.feature_selection import RFE, RFECV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from helpers.feature_selection import select_features_PCA, select_features_RFE

# Step 1: Aggregate the true labels and predicted labels by group
def aggregate_by_group(labels, groups):
    aggregated = {}
    for group, label in zip(groups, labels):
        if group not in aggregated:
            aggregated[group] = []
        aggregated[group].append(label)
    return aggregated

# Step 2: Majority vote within each group
def majority_vote(aggregated):
    for group, labels in aggregated.items():
        most_common = Counter(labels).most_common(1)[0][0]
        aggregated[group] = most_common
    return aggregated

healthy_file = 'D:\Codes\Python\FacialAsymmetry\data\working\csv-single\\Happy-HC-distance.csv'
parkinson_file = 'D:\Codes\Python\FacialAsymmetry\data\working\csv-single\\Happy-PD-distance.csv'

control_df = pd.read_csv(healthy_file)
parkinson_df = pd.read_csv(parkinson_file)

parkinson_df = parkinson_df.assign(label=1)
control_df = control_df.assign(label=0)

data = pd.concat([parkinson_df, control_df], axis=0)
data = data.sample(frac=1,random_state=4)

#Create dataset for train
#data = data.set_index('Person_Data')
data = data.set_index('Image Name')
X = data.drop(['label'], axis=1)
#X = X.drop(['Label'], axis=1)
y = data['label']
X = X.fillna(method='backfill')
X_scaled = X
#selector = RFECV(estimator=LogisticRegression(), step=1, cv=5)
selector = RFE(estimator=LogisticRegression(), n_features_to_select=10)
X_scaled = selector.fit_transform(X_scaled, y)

X_index = data.index.tolist()
y_values = y.tolist()  # Convert y to a list if it's not already

# Create a unique identifier for each sample by combining y with X_index
groups = [f"{y_val}_{x.split('_frame_')[0]}" for y_val, x in zip(y_values, X_index)]
#gridsearch_classifiers_group(X,y,groups)
model_placeholder = Pipeline([
    ('model', LogisticRegression())  # Placeholder for the model
])

# Define the parameter grid
param_grid = [
    # Logistic Regression
    {
        'model': [LogisticRegression(max_iter=10000)],
        'model__C': [0.1, 1,10],
    },
    # Random Forest Classifier
    {
        'model': [RandomForestClassifier()],
        'model__n_estimators': [100, 200],
        'model__max_features': ['log2', 'sqrt'],
    },
    #SVC
    {
        'model': [SVC()],
        'model__C': [0.1,0.5, 1,2, 10],
        'model__kernel': ['linear', 'rbf'],
    },
    #DecisionTreeClassifier
    {
        'model': [DecisionTreeClassifier()],
        'model__max_depth': [5,10,15],
    },
    # XGBoost Classifier
    {
        'model': [XGBClassifier(use_label_encoder=False, eval_metric='logloss')],
        'model__n_estimators': [200],
        'model__learning_rate': [0.01, 0.1, 0.2],
        'model__max_depth': [4],
        'model__subsample': [0.8],  # Subsample ratio of the training instances
        'model__colsample_bytree': [0.9],  # Subsample ratio of columns when constructing each tree
    },
    
]

grid_search = GridSearchCV(model_placeholder, param_grid,  cv=GroupKFold(n_splits=5).split(X, y, groups), scoring='accuracy', verbose=1)
grid_search.fit(X_scaled, y)

# Now, to access the best estimator and its parameters:
best_model = grid_search.best_estimator_
print("Best parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Get unique groups
unique_groups = np.unique(groups)

# Split the unique groups into training and test sets
train_groups, test_groups = train_test_split(unique_groups, test_size=0.2, random_state=8)

# Create boolean masks for training and test sets
train_mask = np.isin(groups, train_groups)
test_mask = np.isin(groups, test_groups)

# Create training and test sets using the boolean masks
X_train, y_train = X_scaled[train_mask], y[train_mask]
X_test, y_test = X_scaled[test_mask], y[test_mask]


clf = best_model
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

# Print the classification report
diff_indices = np.where(y_test != y_pred)[0]
original_indices = y_test.index[diff_indices]
print(sorted(original_indices))
filtered_groups = [group for group, mask in zip(groups, test_mask) if mask]

true_aggregated = aggregate_by_group(y_test, filtered_groups)
pred_aggregated = aggregate_by_group(y_pred, filtered_groups)

true_majority_vote = majority_vote(true_aggregated)
pred_majority_vote = majority_vote(pred_aggregated)

# Step 3: Evaluate the classifier based on the majority-voted labels
y_true_majority = np.array(list(true_majority_vote.values()))
y_pred_majority = np.array(list(pred_majority_vote.values()))

print("Classification report based on majority-voted labels:")
print(classification_report(y_true_majority, y_pred_majority))

Fitting 5 folds for each of 23 candidates, totalling 115 fits
Best parameters: {'model': DecisionTreeClassifier(), 'model__max_depth': 5}
Best Score: 0.6941025641025641
['036dfb9b-b3e1-d827-acb4-c37dae386d9d-joy_frame_24.jpg', '13fc5999-8f79-1b7f-d55b-7c7b8a1a8fcc-joy_frame_89.jpg', '17202_20230718_facial_expression_happy_frame_268.jpg', '28637_20230912_facial_expression_happy_frame_341.jpg', '28731_20230724_facial_expression_happy_frame_353.jpg', '31437_20230612_facial_expression_happy_frame_71.jpg', '31769_20230619_facial_expression_happy_frame_135.jpg', '35246_20230724_facial_expression_happy_frame_238.jpg', '36407_20230612_facial_expression_happy_frame_242.jpg', '38100_20230725_facial_expression_happy_frame_235.jpg', '690916b6-6abf-3a88-a930-fd470096020d-joy_frame_104.jpg', 'cropped_18_Face_1_frame_311.jpg', 'cropped_24_Face_5_frame_135.jpg', 'cropped_27_Face_5_frame_325.jpg', 'cropped_29_Face_1_frame_7.jpg', 'cropped_33_Face_1_frame_10.jpg']
Classification report based on majority

In [16]:
std_of_best_score = grid_search.cv_results_['std_test_score'][grid_search.best_index_]
print("Standard Deviation of Best Score:", std_of_best_score)

Standard Deviation of Best Score: 0.08136269557620797


In [17]:
import numpy as np
indexes = np.where(y_pred_majority != y_true_majority)
#print user column of y_test where the prediction is wrong


In [1]:
import os
from deepface import DeepFace
import shutil

def find_peak_smile_frame_in_all_folders(root_source_folder, destination_folder):
    # Ensure the destination folder exists
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    
    # Iterate through each sub-folder in the root source folder
    for folder_name in os.listdir(root_source_folder):
        source_folder = os.path.join(root_source_folder, folder_name)
        if os.path.isdir(source_folder):  # Make sure it's a folder
            highest_smile_score = 0
            peak_smile_frame = None

            # Iterate through each file in the current source folder
            for filename in os.listdir(source_folder):
                if filename.endswith(".jpg") or filename.endswith(".png"):  # Check for image files
                    try:
                        # Construct the full file path
                        file_path = os.path.join(source_folder, filename)

                        # Analyze the facial expression in the image
                        analysis = DeepFace.analyze(img_path=file_path, actions=['emotion'])

                        # Check if the 'happy' score is the highest we've seen
                        smile_score = analysis[0]['emotion']['happy']
                        if smile_score > highest_smile_score:
                            highest_smile_score = smile_score
                            peak_smile_frame = file_path

                    except Exception as e:
                        print(f"Error processing {filename}: {e}")

            # If a peak smile frame was found, copy it to the destination folder with a new name
            if peak_smile_frame:
                # Construct new filename as [folder_name]_[original_filename]
                new_filename = f"{folder_name}_{os.path.basename(peak_smile_frame)}"
                new_file_path = os.path.join(destination_folder, new_filename)
                shutil.copy(peak_smile_frame, new_file_path)
                print(f"Peak smile frame '{peak_smile_frame}' copied to '{new_file_path}'.")

# Define your root source folder where all sub-folders are located
root_source_folder = 'D:\Codes\Python\FacialAsymmetry\data\working\\aligned\PD'
# Define the destination folder where you want to copy the peak smile frames
destination_folder = 'D:\Codes\Python\FacialAsymmetry\data\working\\single-happy\PD'

# Execute the function
find_peak_smile_frame_in_all_folders(root_source_folder, destination_folder)




Peak smile frame 'D:\Codes\Python\FacialAsymmetry\data\working\aligned\PD\cropped_05_Face_1\frame_77.jpg' copied to 'D:\Codes\Python\FacialAsymmetry\data\working\single-happy\PD\cropped_05_Face_1_frame_77.jpg'.
Peak smile frame 'D:\Codes\Python\FacialAsymmetry\data\working\aligned\PD\cropped_06_Face_1\frame_29.jpg' copied to 'D:\Codes\Python\FacialAsymmetry\data\working\single-happy\PD\cropped_06_Face_1_frame_29.jpg'.
Peak smile frame 'D:\Codes\Python\FacialAsymmetry\data\working\aligned\PD\cropped_07_Face_1\frame_9.jpg' copied to 'D:\Codes\Python\FacialAsymmetry\data\working\single-happy\PD\cropped_07_Face_1_frame_9.jpg'.
Peak smile frame 'D:\Codes\Python\FacialAsymmetry\data\working\aligned\PD\cropped_10_Face_1\frame_93.jpg' copied to 'D:\Codes\Python\FacialAsymmetry\data\working\single-happy\PD\cropped_10_Face_1_frame_93.jpg'.
Peak smile frame 'D:\Codes\Python\FacialAsymmetry\data\working\aligned\PD\cropped_12_Face_1\frame_161.jpg' copied to 'D:\Codes\Python\FacialAsymmetry\data\wo

SPIGA model loaded!


d:\Codes\Python\FacialAsymmetry\venv\lib\site-packages\torch\nn\functional.py:4298: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
d:\Codes\Python\FacialAsymmetry\venv\lib\site-packages\torch\nn\functional.py:4236: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


: 